### **Step 1: Install and Import Libraries**

In [ ]:
pip install google-play-scraper nltk

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import nltk
import re

from google_play_scraper import Sort, reviews_all
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt')
nltk.download("vader_lexicon")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\WT\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

### **Step 2: Scrape User Reviews from Google Play Store**

In [ ]:
app_reviews = reviews_all(
        "my.com.tngdigital.ewallet",
        sleep_milliseconds=0, # defaults to 0
        lang='en', # defaults to 'en'
        country='us', # defaults to 'us'
        sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
    )

In [ ]:
df = pd.DataFrame(np.array(app_reviews),columns=['review'])
df2 = df.join(pd.DataFrame(df.pop('review').tolist()))

In [ ]:
reviews_df = df2[['content', 'score']]